# 0. Variabili globali e funzioni di utils (da eseguire ogni volta)

In [4]:
import torch
from torchvision import transforms
import os
import csv
import numpy as np
from PIL import Image
from facenet_pytorch.models.inception_resnet_v1 import InceptionResnetV1
from sklearn.metrics import accuracy_score


# pre-processa le immagini in modo compatibile con la rete.
def load_images(filepath):
    img = Image.open(filepath).convert("RGB")
    rsz = img.resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns, rsz


def evaluate_performance(device, net, test_set=".\\dataset\\test_set", labels=".\\dataset\\test_set.csv"):
    y_true = []
    y_pred = []
    LABELS = np.load(".\\dataset\\rcmalli_vggface_labels_v2.npy")
    true_labels = {str(name).strip(): idx for idx, name in enumerate(LABELS)}

    with open(labels, "r", encoding="utf-8") as csvfile: # apri con la codifica corretta.
        reader = csv.reader(csvfile)

        for row in reader:
            filename, name = row[0], row[1].strip(' "') # togli spazio e doppi apici.
            img_path = os.path.join(test_set, filename)
            for img in os.listdir(img_path):
                try:
                    img_tensor, _ = load_images(img_path + "/" + img) # usa il path completo.
                except Exception as e:
                    print(f"Errore nel caricamento di {img_path}: {e}")
                    continue

                img_tensor = img_tensor.unsqueeze(0)  # aggiunge dimensione batch.
                with torch.no_grad():
                    output = net(img_tensor.to(device))
                    # dato l'output ottieni la stringa corrispondente.
                    predicted_class = np.array(output[0].detach().cpu().numpy()).argmax()

                y_true.append(true_labels[name])
                y_pred.append(predicted_class)

    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
NN1 = InceptionResnetV1(pretrained='vggface2').eval()
NN1.to(device)
NN1.classify = True
print("Modello NN1 caricato correttamente")

Using device: cuda
Modello NN1 caricato correttamente


# 1. Creazione del test set


In [5]:
import os
import csv
import shutil
import random

# Parametri
csv_file = '.\\dataset\\test_set.csv' # Percorso al file CSV contenente gli ID delle persone da inserire nel dataset finale (es. 'n000016'). Ogni riga deve contenere almeno un ID
dataset_directory_origin = '.\\dataset\\vggface2_train\\train' # Directory di origine contenente le sottocartelle per ogni ID persona con le immagini
dataset_directory_destination = '.\\dataset\\test_set' # Directory in cui salvare le immagini selezionate
number_img = 10 # Numero massimo di immagini da copiare per ciascun ID

# Lettura del file CSV contenente gli ID delle persone
with open(csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')  # i campi sono separati da virgole

    for row in reader:
        if len(row) < 1:
            continue  # salta righe vuote

        person_id = row[0].strip()  # es: "n000016"
        person_name = row[1].strip(' "')

        # Costruzione dei percorsi di origine e destinazione per ogni persona
        origin_path = os.path.join(dataset_directory_origin, person_id)
        destination_path = os.path.join(dataset_directory_destination, person_id)

        if not os.path.exists(origin_path):
            print(f"Cartella non trovata: {origin_path}")
            continue

        # Lista dei file immagine nella cartella della persona
        images = [f for f in os.listdir(origin_path) if os.path.isfile(os.path.join(origin_path, f))]
        if not images:
            print(f"Nessuna immagine in: {origin_path}")
            continue

        # Seleziona un numero massimo di immagini in modo casuale
        random.seed(2025)
        selected_images = random.sample(images, min(number_img, len(images)))

        # Crea la cartella di destinazione se non esiste
        os.makedirs(destination_path, exist_ok=True)

        # Copia e rinomina le immagini nella cartella di destinazione
        for idx, image in enumerate(selected_images):
            src = os.path.join(origin_path, image)
            ext = os.path.splitext(image)[1]  # estensione del file (es. .jpg)
            dst_filename = f"{person_id}_{idx + 1:02d}{ext}"  # es: n000016_01.jpg
            dst = os.path.join(destination_path, dst_filename)
            shutil.copy2(src, dst)

        print(f"Copiate {len(selected_images)} immagini per {person_name}")

Copiate 10 immagini per Paolo_Maldini
Copiate 10 immagini per Kimi_Räikkönen
Copiate 10 immagini per Antonio_Candreva
Copiate 10 immagini per Antonio_Cassano
Copiate 10 immagini per Gerry_Scotti
Copiate 10 immagini per Stephen_Curry
Copiate 10 immagini per Michael_B._Jordan
Copiate 10 immagini per Kobe_Bryant
Copiate 10 immagini per LeBron_James
Copiate 10 immagini per Francesco_Totti
Copiate 10 immagini per Alessandro_Nesta
Copiate 10 immagini per Alexandre_Pato
Copiate 10 immagini per Michael_Schumacher
Copiate 10 immagini per Katie_Perry
Copiate 10 immagini per Michael_Ballack
Copiate 10 immagini per Gigi_DAlessio
Copiate 10 immagini per Gianluca_Zambrotta
Copiate 10 immagini per Gianluigi_Buffon
Copiate 10 immagini per Bob_Marley
Copiate 10 immagini per Marek_Hamšík
Copiate 10 immagini per Adrian_Newey
Copiate 10 immagini per Cristiano_Ronaldo
Copiate 10 immagini per Alessio_Cerci
Copiate 10 immagini per Alessandro_Del_Piero
Copiate 10 immagini per Alessandra_Amoroso
Copiate 10 imm

# 2. Calcolo delle performance sui dati clean

In [6]:
evaluate_performance(device, NN1)

Accuracy: 0.824
